In [16]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from datetime import datetime

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
all_predictions = []
folder_path = '../data/predictions/'
files = os.listdir(folder_path)
# drop non-csv files (e.g. DS_Store)
files = [file for file in files if file.endswith('.csv')]
for file in files:
    predictions_week = pd.read_csv(folder_path + file, index_col=0)
    creation_time = os.path.getctime(folder_path + file)
    creation_date = datetime.fromtimestamp(creation_time).date()
    predictions_week['prediction_date'] = creation_date
    all_predictions.append(predictions_week)
len(all_predictions)

In [ ]:
player_name = 'Haaland'
player_predictions = pd.DataFrame()
for week in range(len(all_predictions)):
    new_player_predictions = (all_predictions[week][all_predictions[week]
                                .name.str.contains(player_name)].copy())
    player_predictions = pd.concat((player_predictions, new_player_predictions))
player_predictions

In [ ]:
player_predictions.columns.tolist()

In [ ]:
player_predictions[['prediction_date', 'gameweek', 'opponent_team', 'expected_points']]

In [ ]:
grouped = (player_predictions[['prediction_date', 'gameweek', 'opponent_team', 'expected_points']]
           .sort_values(by='prediction_date')
           .groupby(['gameweek', 'opponent_team']))
grouped.groups

In [ ]:
for name, group in grouped:
    print(f"Group: {name}")
    print(group)

In [ ]:
for name, group in grouped:
    print(f"Group: {name}")
    fig = px.line(x=group.prediction_date, y=group.expected_points, markers=True)
    fig.show()

In [ ]:
subtitles = []
for name, group in grouped:
    subtitles.append('Week ' + str(name[0]) + ' ' +name[1])
subtitles

In [ ]:
nr_figs = len(grouped.groups)
fig = make_subplots(rows=nr_figs, cols=1, subplot_titles=subtitles)

i=1
for name, group in grouped:
    fig.append_trace(go.Scatter(
        x=group.prediction_date,
        y=group.expected_points,
        showlegend=False,
    ), row=i, col=1)
    i+=1

fig.update_layout(height=100*nr_figs, width=600)
fig.show()

In [ ]:
mean_predictions = (player_predictions[['prediction_date', 'expected_points']]
           .groupby(['prediction_date'])).mean()
mean_predictions

In [ ]:
px.line(x=mean_predictions.index, y=mean_predictions.expected_points, markers=True)